In [ ]:
# default_exp data_man
%reload_ext autoreload
%autoreload 2

# Data management
> Create, from FT1 and FT2, a compact data set with photon and livetime info.

### Overview

Fermi-LAT weekly data files are extracted from the [GSFC FTP server ](https://heasarc.gsfc.nasa.gov/FTP/fermi/data/lat/weekly), 
with subfolders for the photon data, `photon` and spacecraft data, `spacecraft`. It is [described here](http://fermi.gsfc.nasa.gov/ssc/data/access/http://fermi.gsfc.nasa.gov/ssc/data/access/)

The class `FermiData` downloads these to temporary files and constructs a dict for each week with
contents

* photons: a table, entry per selected photon with columns, converted with `get_ft1_data`

  * run number (uint32, stored as a category by pandas)
  * time since the run start, in 2 $\mu$s intervals  (uint32)
  * energy and event type (uint8)
  * position as HEALPix index (uint32)
  
* sc_data: a table, an entry per 30-s interval, with columns, all float32, converted with `get_ft2_info`
  * start/stop time 
  * S/C direction 
  * zenith direction
* gti_times: an array of interleaved start/stop intervals
* file_date: modification date for the FT1 file at GSFC.

These dict objects, one per week, are saved in a folder

#### A note about timing
A run is typically an orbit or less, at most 6 ks. Integerizing the offset from this using 32 bits, one has 5e5 intervals/s, so 
we choose 2$\mu$s. 

In [ ]:
from nbdev.showdoc import *

In [ ]:
# export
import os, sys
import dateutil, datetime
from astropy.io import fits
from ftplib import FTP_TLS as FTP

import healpy
from pathlib import Path
import numpy as np
import pandas as pd

import pickle

from wtlike.config import Config, Timer, UTC, MJD

In [ ]:
# export
def get_ft1_data( config, ft1_file):

        """
        Read in a photon data (FT1) file, bin in energy and position to convert to a compact DataFrame

        - `ft1_file` -- A monthly file generated by J. Ballet, or a weekly file from GSFC

        Depends on config items
        - `theta_cut, z_cut` -- selection criteria
        - `ebins, etypes` -- define band index
        - `nside, nest` -- define HEALPix binning

        Returns a tuple with

        - `tstart`, the start MET time

        - DataFrame  with columns
           - `band` (uint8):    energy band index*2 + 0,1 for Front/Back
           - `nest_index`  if nest else `ring_index` (uint32): HEALPIx index for the nside
           - `run_id` (uint32) The run number, stored as a categorical uint32 array
           - `trun` (unit32): time since run the id in 2 $\mu s$ units

        - gti times as an interleaved start, stop array.

        For the selected events above 100 MeV, this represents 9 bytes per photon, vs. 27.
        """

        delta_t = config.offset_size
        ebins = config.energy_edges
        etypes = config.etypes
        nside = config.nside
        nest = config.nest
        z_cut =config.z_max
        theta_cut = np.degrees(np.arccos(config.cos_theta_max))
        verbose = config.verbose

        with  fits.open(ft1_file) as ft1:
            tstart = ft1[1].header['TSTART']

            ## GTI - setup raveled array function to make cut
            gti_data= ft1['GTI'].data
            # extract arrays for values of interest
            data =ft1['EVENTS'].data

        a,b = sorted(gti_data.START), sorted(gti_data.STOP)

        gti_times = np.ravel(np.column_stack((a,b)))
        if np.any(np.diff(gti_times)<0):
            raise Exception(f'Non-monatonic GTI found in file {ft1_file}')

        # def apply_gti(time):
        #     x = np.digitize(time, gti_times)
        #     return np.bitwise_and(x,1).astype(bool)

        # apply  selections

        sel =  ((data['ENERGY'] > ebins[0]) &
                (data['ZENITH_ANGLE'] < z_cut) &
                (data['THETA'] < theta_cut))

        dsel = data[sel]

        # get the columns for output
        glon, glat, energy, et, z, theta, time, ec =\
             [dsel[x] for x in 'L B ENERGY EVENT_TYPE ZENITH_ANGLE THETA TIME EVENT_CLASS'.split()]

        # generate event_type masks
        et_mask={}
        for ie in etypes:
            et_mask[ie]= et[:,-1-ie]


        if verbose>1:
            total = sum(b)-sum(a)
            fraction = total/(b[-1]-a[0])

            print(  f'FT1: {ft1_file.name}, GTI range {a[0]:.1f}-{b[-1]:.1f}:  {len(data):,} photons'\
                    f'\n\tSelection E > {ebins[0]:.0f} MeV. theta<{theta_cut:.1f} and z<{z_cut} remove:'\
                    f' {100.- 100*len(dsel)/float(len(data)):.2f}%'
#                     f', GTI cut removes {sum(~gti_cut)}'
                 )


        # event class -- turn into single int for later mask
#         bits = np.array([1<<n for n in range(20)])
#         def to_bin(x):
#             return np.sum(bits[x[:20]])
#         ec = [to_bin(row[20]) for row in ec

        # pixelate direction
        hpindex = healpy.ang2pix(nside, glon, glat, nest=nest, lonlat=True).astype(np.uint32)
        hpname = 'nest_index' if nest else 'ring_index'

        # digitize energy and create band index incluing (front/back)
        band_index = (2*(np.digitize(energy, ebins, )-1) + et_mask[1]).astype(np.uint8)

        #
        run_id = dsel['RUN_ID'].astype(np.uint32)


        df = pd.DataFrame(
                {   'band'  : band_index,
                    hpname  : hpindex,
                    #'time'  : (time-tstart).astype(np.float32), # the old time
                    'run_id': pd.Categorical(run_id),
                    'trun'  : ((time-run_id)/delta_t).astype(np.uint32),
                }  )
        if verbose>1:
            print(f'\tReturning tstart={tstart:.0f}, {len(dsel):,} photons.')

        return  tstart, df, gti_times

In [ ]:
show_doc(get_ft1_data, title_level=2)

<h2 id="get_ft1_data" class="doc_header"><code>get_ft1_data</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h2>

> <code>get_ft1_data</code>(**`config`**, **`ft1_file`**)

Read in a photon data (FT1) file, bin in energy and position to convert to a compact DataFrame

- `ft1_file` -- A monthly file generated by J. Ballet, or a weekly file from GSFC

Depends on config items
- `theta_cut, z_cut` -- selection criteria
- `ebins, etypes` -- define band index
- `nside, nest` -- define HEALPix binning

Returns a tuple with

- `tstart`, the start MET time

- DataFrame  with columns
   - `band` (uint8):    energy band index*2 + 0,1 for Front/Back
   - `nest_index`  if nest else `ring_index` (uint32): HEALPIx index for the nside
   - `run_id` (uint32) The run number, stored as a categorical uint32 array
   - `trun` (unit32): time since run the id in 2 $\mu s$ units

- gti times as an interleaved start, stop array.

For the selected events above 100 MeV, this represents 9 bytes per photon, vs. 27.

In [ ]:
# export
def get_ft2_info(config, filename,
                 gti=lambda t: True):
    """Process a FT2 file, with S/C history data, and return a summary DataFrame

    Parameters:

    * config -- verbose, cos_theta_max, z_max
    * filename -- spacecraft (FT2) file
    * gti -- GTI object that checkes for allowed intervals, in MJD units

    Returns: A DataFrame with fields consistent with GTI if specified

    * start, stop -- interval in MJD units
    * livetime -- sec
    * ra_scz, dec_scz --spaceraft direction
    * ra_zenith, dec_zenith -- local zenith
    """
    # combine the files into a DataFrame with following fields besides START and STOP (lower case for column)
    fields    = ['LIVETIME','RA_SCZ','DEC_SCZ', 'RA_ZENITH','DEC_ZENITH']
    with fits.open(filename) as hdu:
        scdata = hdu['SC_DATA'].data
        tstart, tstop = [float(hdu[0].header[field]) for field in  ('TSTART','TSTOP') ]

    if config.verbose>1:
        print(f'FT2: {filename.name}, MET range {tstart:.1f}-{tstop:.1f},', end='')# {"not" if gti is None else ""} applying GTI')

    # get times to check against MJD limits and GTI
    start, stop = [MJD(np.array(scdata.START, float)),
                   MJD(np.array(scdata.STOP, float))]

    # apply GTI to bin center (avoid edge effects?)
    in_gti = gti(0.5*(start+stop))
    if config.verbose>1:
        s = sum(in_gti)
        print(f' {len(start)} entries, {s} ({100*s/len(start):.1f}%) in GTI')

    t = [('start', start[in_gti]), ('stop',stop[in_gti])]+\
        [(field.lower(), np.array(scdata[field][in_gti],np.float32)) for field in fields ]

    sc_data = pd.DataFrame(dict(t) )

    return sc_data

In [ ]:
show_doc(get_ft2_info, title_level=2)

<h2 id="get_ft2_info" class="doc_header"><code>get_ft2_info</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h2>

> <code>get_ft2_info</code>(**`config`**, **`filename`**, **`gti`**=*`<lambda>`*)

Process a FT2 file, with S/C history data, and return a summary DataFrame

Parameters:

* config -- verbose, cos_theta_max, z_max
* filename -- spacecraft (FT2) file
* gti -- GTI object that checkes for allowed intervals, in MJD units

Returns: A DataFrame with fields consistent with GTI if specified

* start, stop -- interval in MJD units
* livetime -- sec
* ra_scz, dec_scz --spaceraft direction
* ra_zenith, dec_zenith -- local zenith

In [ ]:
# export
def filepaths(week):
    """Returns: A tuple with two elements for the week number, each with two triplets with:
        ftp folder, ftp filename, local simple filename
    """
    urls = []
    for ftype, alias in  [('spacecraft','ft2'), ('photon','ft1')]:
         urls.append((
             f'{ftype}',
             f'lat_{ftype}_weekly_w{week:03d}_p{"305" if ftype=="photon" else "310" }_v001.fits',
             f'week{week:03d}_{alias}.fits',
            ))
    return urls

In [ ]:
# export
class FermiData(dict):
    """ Manage the full data set in weekly chunks
    * Checking the current set of files at GSFC
    * downloading a week at a time to a local tmp
    * Converting to condensed format and saving to pickled dicts in wtlike_data
    """

    ftp_site = 'heasarc.gsfc.nasa.gov'
    ftp_path = 'fermi/data/lat/weekly'
    local_path  = '/tmp/from_gsfc'

    def __init__(self, config=None):
        """ Obtain list of the weekly FT1 and FT2 files at GSFC, Set up as a dict, with
        keys= week numbers, values=mofification date strings
        """
        self.config = config or Config()
        self.wtlike_data_file_path = Path(self.config.datapath/'data_files')
        assert self.wtlike_data_file_path.is_dir(), 'Data path invalid'
        os.makedirs(self.local_path, exist_ok=True)

        try:
            with FTP(self.ftp_site) as ftp:
                ftp.login()
                ftp.prot_p()
                ftp.cwd(self.ftp_path+'/photon') # or spacecraft
                # aet modification time and type for all files in folder

                parse_week = lambda fn:  int(fn.split('_')[3][1:])
                flist = ftp.mlsd(facts=['modify', 'type'])
                self.fileinfo = sorted([(parse_week(name), fact['modify']) for name,fact in flist
                                 if fact['type']=='file' and name.startswith('lat') ])
        except Exception as msg:
            raise Exception(f'FTP login to or download from {self.ftp_site} failed: {msg}')
        self.update(self.fileinfo)

    @property
    def local_filedate(self):
        """ the datetime object representing the last file date in local storage"""
        from dateutil.parser import parse
        weekly_folder = self.config.datapath/'data_files'
        ff = sorted(list(weekly_folder.glob('*.pkl')))
        if len(ff)==0:
            print(f'No .pkl files found in {weekly_folder}', file=sys.stderr)
            return None

        wk = list(map(lambda f: int(os.path.splitext(f)[0][-3:]), ff))
        lastweek = pickle.load(open(ff[-1],'rb'))
        return dateutil.parser.parse(lastweek['file_date'])

    @property
    def gsfc_filedate(self):
        return dateutil.parser.parse(list(self.values())[-1])

    def download(self, week):
        """ Download the given week to the tmp folder
        """
        assert week in self, f'week {week} not found at FTP site'
        with FTP(self.ftp_site) as ftp:
            ftp.login()
            ftp.prot_p()
            for ftp_folder, ftp_filename, local_filename in filepaths(week):
                ftp.cwd('/'+self.ftp_path+'/'+ftp_folder)
                if self.config.verbose>0:
                    print(f'FermiData: {ftp_folder}/{ftp_filename} --> {local_filename}')
                with open(f'{self.local_path}/{local_filename}', 'wb') as  localfile:
                    ftp.retrbinary('RETR ' + ftp_filename, localfile.write)

    def __str__(self):
        return f'FermiData: {len(self.fileinfo)} week files at GSFC, from {self.fileinfo[0]} to {self.fileinfo[-1]}'

    def in_temp(self):
        """return list of GSFC copied files in the local_path folder"""
        names = [f.name for f in Path(self.local_path).glob('*')]
        return names

    def __call__(self, week, test=False, tries_left=2):
        """ Process the given week:
        * download from GSFC
        * convert each
        * save pickled dict summary

        """
        assert week in self, f'week {week} not found at FTP site'
        ff = filepaths(week)
        ft1_file = Path(self.local_path)/ff[1][2]
        ft2_file = Path(self.local_path)/ff[0][2]

        if self.config.verbose>1:
            print(f'FermiData: converting week {week}')

        while tries_left>0:
            try:
                if not (ft1_file.exists() and ft2_file.exists()):
                    self.download(week)
                tstart, photon_data, gti_times  = get_ft1_data(self.config, ft1_file)
                break
            except Exception as e:
                print(f'*** ERROR *** Failed to convert {ft1_file}: {e} download it again)')
                tries_left -=1
                if tries_left==0:
                    print(f'Failed to convert week file {ft1_file}: quitting', file=sys.stderr)
                    return None
                else:
                    os.unlink(ft1_file)

        def apply_gti(time): # note MJD
            x = np.digitize(time, MJD(gti_times))
            return np.bitwise_and(x,1).astype(bool)
        sc_data = get_ft2_info(self.config, ft2_file, apply_gti)

        # finished with copies of FT1 and FT2 files: delete them
        for file in (ft1_file,ft2_file):
            os.unlink(file)

        # package info into a dict for pickle
        d = dict(tstart = tstart,
                photons = photon_data,
                sc_data = sc_data,
                gti_times = gti_times,
                file_date = self[week])
        filename = self.wtlike_data_file_path/f'week_{week:03d}.pkl'
        if filename.exists():
            print(f'FermiData: replacing existing {filename}')
        if not test:
            with open(filename, 'wb') as out:
                pickle.dump(d, out)

        if self.config.verbose>0:
            print(f'FermiData: Saved to {filename}')
            if self.config.verbose>1:
                print(photon_data.info())

    def load_week(self, week):
        """Load a pickled week summary """
        filename = self.wtlike_data_file_path/f'week_{week:03d}.pkl'
        assert filename.exists(), f'File {filename} does not exist'
        with open(filename, 'rb') as imp:
            ret = pickle.load(imp)
        return ret

    def check_week(self, week):
        """Returns True if the local week needs updating"""
        data = self.load_week(week)
        if 'file_date' not in data:
            return True
        return data['file_date'] == self[week]

    def needs_update(self, threshold=0):
        """ Compare files on disk with the GSFC list and compile list that need to be downloaded

        Check the file date of the last one on disk and include it if:
            * it short and there is one or more GSFC weeks following it,
            * It is the most recent week and is short by more than *threshold* days
        """
        gg =self.wtlike_data_file_path.glob('*.pkl')

        file_weeks= map(lambda n: int(n.name[5:8]), gg)
        ondisk = np.array(list(file_weeks))

        missing =  list(set(self.keys()).difference(set(ondisk)))

        last = ondisk[-1]
        if last not in missing and not self.check_week( last):
            delta = (self.gsfc_filedate -self.local_filedate).seconds/24/3600
            if delta> threshold:
                missing.append(last)
        return missing

    def process(self, days=1):
        """ Download and process all weeks missing or needing an update, if within `days`
        from the present

        Return status: True if  anything changed
        """
        # will use multprocessing if len(todo)>1 and pool_size>1
        todo = self.needs_update(days)
        if len(todo)==0: return False
        if self.config.pool_size >1 and len(todo)>1:
            print('multitasking not applied yet', file=sys.stderr)
            pass
        list(map(self, todo))
        return True

### FermiData methods

In [ ]:
show_doc(FermiData.download)

<h4 id="FermiData.download" class="doc_header"><code>FermiData.download</code><a href="__main__.py#L55" class="source_link" style="float:right">[source]</a></h4>

> <code>FermiData.download</code>(**`week`**)

Download the given week to the tmp folder
        

In [ ]:
show_doc(FermiData.needs_update)

<h4 id="FermiData.needs_update" class="doc_header"><code>FermiData.needs_update</code><a href="__main__.py#L149" class="source_link" style="float:right">[source]</a></h4>

> <code>FermiData.needs_update</code>(**`threshold`**=*`0`*)

Compare files on disk with the GSFC list and compile list that need to be downloaded

Check the file date of the last one on disk and include it if:
    * it short and there is one or more GSFC weeks following it,
    * It is the most recent week and is short by more than *threshold* days

In [ ]:
self = FermiData()
print(self, self.needs_update())

FermiData: 711 week files at GSFC, from (9, '20220228224421') to (720, '20220317161457') [720, 717, 718, 719, 716]


In [ ]:
show_doc(FermiData.process)

<h4 id="FermiData.process" class="doc_header"><code>FermiData.process</code><a href="__main__.py#L170" class="source_link" style="float:right">[source]</a></h4>

> <code>FermiData.process</code>(**`days`**=*`1`*)

Download and process all weeks missing or needing an update, if within `days`
from the present

Return status: True if  anything changed

In [ ]:
show_doc(FermiData.check_week)

<h4 id="FermiData.check_week" class="doc_header"><code>FermiData.check_week</code><a href="__main__.py#L142" class="source_link" style="float:right">[source]</a></h4>

> <code>FermiData.check_week</code>(**`week`**)

Returns True if the local week needs updating

In [ ]:
config = Config()
if config.valid:
    self = FermiData(Config(verbose=1))
    check =self.needs_update(0.5)
    print(self, f'\n\tweek(s) needing update: {check}' )

FermiData: 711 week files at GSFC, from (9, '20220228224421') to (720, '20220317161457') 
	week(s) needing update: [720, 717, 718, 719, 716]


In [ ]:
#hide
#     with Timer() as t:
#         self.process(0.5)
#     print(t)

## check the weekly files

In [ ]:
# export
def check_data(config=None):
    """
    Return: sorted list of files, last week number, number of days in last week"""
    if config is None: config=Config()
    if config.valid:
        weekly_folder = config.datapath/'data_files'
        ff = sorted(list(weekly_folder.glob('*.pkl')))
        if len(ff)==0:
            print(f'No .pkl files found in {weekly_folder}', file=sys.stderr)
            return
        wk = list(map(lambda f: int(os.path.splitext(f)[0][-3:]), ff))
        lastweek = pickle.load(open(ff[-1],'rb'))
        file_date = lastweek['file_date']
        gti = lastweek['gti_times'];
        days = (gti[-1]-gti[0])/(24*3600)
        if config.verbose>0:
            print(f'Weekly folder "{weekly_folder}" contains {len(wk)} weeks.'\
                  f'\n\t Last week, # {wk[-1]}, has {days:.3f} days, ends at UTC {UTC(MJD(gti[-1]))}, filedate {file_date}' )
        return ff, wk[-1], days
    else:
        print(f'Config not valid, {config.errors}', file=sys.stderr)
        return None

In [ ]:
#ret = check_data()

In [ ]:
# export
def update_data(update_threshold=1, config=None):
    """Bring all of the local week data summaries up to date, downloading the missing ones from GSFC.
    If the last one is the current week, check to see if needs updating, by comparing file date, in days,
    from the last update with the current one at GSFC.
    """
    ff = FermiData(config)
    return ff.process(update_threshold)

In [ ]:
#export
def get_week_files(config, week_range=None):
    """Return list of week files

    - week_range [None] -- tuple with inclusive range. If None, get all
    """
    data_folder = config.datapath/'data_files'
    data_files = sorted(list(data_folder.glob('*.pkl')))
    weeks = week_range or  config.week_range
    if week_range is not None:

        slc = slice(*week_range) if type(week_range)==tuple else slice(week_range,week_range)
        wk_table = pd.Series(data=[df for df in data_files],
                     index= [ int(df.name[-7:-4]) for df in  data_files],
                    )
        data_files = wk_table.loc[slc].values

        if config.verbose>0:
            q = lambda x: x if x is not None else ""
            print(f'LoadData: Loading weeks[{q(slc.start)}:{q(slc.stop)}:{q(slc.step)}]', end='' if config.verbose<2 else '\n')
    else:
        if config.verbose>0: print(f'LoadData: loading all {len(data_files)} weekly files')

    if len(data_files)==0:
        msg =  f'Specified week_range {week_range} produced no output. Note that week numbers are 9-'
        raise Exception(msg)

    return data_files

In [ ]:
# hide
from nbdev.export import notebook2script
notebook2script()
!date

Converted 00_config.ipynb.
Converted 01_data_man.ipynb.
Converted 02_effective_area.ipynb.
Converted 03_exposure.ipynb.
Converted 03_sources.ipynb.
Converted 04_load_data.ipynb.
Converted 04_simulation.ipynb.
Converted 05_source_data.ipynb.
Converted 06_poisson.ipynb.
Converted 07_loglike.ipynb.
Converted 08_cell_data.ipynb.
Converted 09_lightcurve.ipynb.
Converted 10-time_series.ipynb.
Converted 14_bayesian.ipynb.
Converted 90_main.ipynb.
Converted 99_presentation.ipynb.
Converted 99_tutorial.ipynb.
Converted index.ipynb.
Thu Mar 17 09:21:00 PDT 2022
